# Implementation of Ditchnet

The raw data is stored as laz in different folders each year. I wanted to split the data wrangling across multiple computers to speed things up and spare the NAS. First check number of laz files within each year. 2018 to 2021

In [2]:
import os
import sys
import subprocess
import glob

Check number of laz files 2018

In [7]:
listoflazfiles2018 = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/2018/**/*.laz', recursive = True)
print(len(listoflazfiles2018),' lazfiles were scanned 2018')
listoflazfiles2019 = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/2019/**/*.laz', recursive = True)
print(len(listoflazfiles2019),' lazfiles were scanned 2019')
listoflazfiles2020 = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/2020/**/*.laz', recursive = True)
print(len(listoflazfiles2020),' lazfiles were scanned 2020')
listoflazfiles2021 = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/2021/**/*.laz', recursive = True)
print(len(listoflazfiles2021),' lazfiles were scanned 2021')


7535  lazfiles were scanned 2018
13734  lazfiles were scanned 2019
13762  lazfiles were scanned 2020
812  lazfiles were scanned 2021


only a handfull of tiles were scanned 2021 so these will be boundled together with 2020. Each computer will download lazfiles for a year and process them seperatly. 
32 GB will do 2018
128 GB will do 2019 F: F:\DitchNet\folders
Threadripper will do 2020 and 2021: D:\DitchNet
Each computer gets a folder named DitchNet



The laz files had to be unpacked to las

In [2]:
import glob
listoflazfiles = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/**/*.laz', recursive = True)
print(len(listoflazfiles),' laz files')

35843  laz files


In [6]:
from __future__ import print_function
import threading
import os
import sys
import subprocess
import glob


lasdir = 'Y:/William/DeepLearning/Implementation/LiDAR/lasfiles/'

maxThreads = 60      #threads
activeThreads = 0

class workerThread(threading.Thread):
    def __init__(self, indata):
        threading.Thread.__init__(self)
        self.file = indata

    def run(self):
        global activeThreads
        activeThreads += 1

        #################################
        ##### Start here #####
        #################################
        global lasdir
        lasout = lasdir + os.path.basename(self.file)

        try:
            subprocess.call(['Y:/William/DeepLearning/Implementation/code/utils/laszip', '-i', self.file, '-o', lasout.replace('.laz','.las')])
            print('unpacked ', self.file)
        except:
            print('Unexpected error:', sys.exc_info()[0])
            raise
        #################################
        ##### Stop here #####
        #################################

        activeThreads -= 1

listoflazfiles = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/**/*.laz', recursive = True)
for file in listoflazfiles:
    worker = workerThread(file).start()
    while activeThreads >= maxThreads:
        pass


35843

In [4]:
import glob
listoflasfiles = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/lasfiles/**/*.las', recursive = True)
print(len(listoflasfiles),' las files')

35843  las files


In [9]:
listoflasfilesTR = glob.glob('D:/DitchNet/lasfiles/**/*.las', recursive = True)
print(len(listoflasfilesTR),' lasfiles are located on threadripper las folder')
print(listoflasfilesTR[0])

22109  lasfiles are located on threadripper las folder
D:/DitchNet/lasfiles\18A003_61475_3675_25.las


LAS to dem got stuck at 92 % so this script was used to copy remining files to a new folder

In [12]:
import sys
import os
import shutil
import glob

listoflasfiles = glob.glob('D:/DitchNet/lasfiles/**/*.las', recursive = True)
listofdemfiles = glob.glob('D:/DitchNet/lasfiles/**/*.tif', recursive = True)
REMAINING = 'D:/DitchNet/remaininglasfiles/'
remaininglist = []
for file in listoflasfiles:
    if file.replace('.las','.tif') not in listofdemfiles:
        outlas =  REMAINING + os.path.basename(file)
        #print(outlas)
        remaininglist.append(outlas)

        shutil.copy(file, outlas)
print(len(remaininglist))



1767


Convert remainingtiles to dem

In [ ]:
import sys
import os
sys.path.insert(1, 'D:/DitchNet/code/utils/WBT/')
from whitebox_tools import WhiteboxTools
whitebox_dir = os.path.dirname('D:/DitchNet/code/utils/WBT/')
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(whitebox_dir)

whitebox.work_dir = 'D:/DitchNet/remaininglasfiles/'
whitebox.lidar_tin_gridding(
    i=None,
    output=None,
    parameter='elevation',
    returns='last',
    resolution=0.5,
    exclude_cls='0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18',
    minz=None,
    maxz=None,
    max_triangle_edge_length=None,
    )


It got stuck at 98 % again

In [14]:
import sys
import os
import shutil
import glob

listoflasfiles = glob.glob('D:/DitchNet/remaininglasfiles/**/*.las', recursive = True)
listofdemfiles = glob.glob('D:/DitchNet/remaininglasfiles/**/*.tif', recursive = True)
REMAINING = 'D:/DitchNet/remaininglasfiles2/'
remaininglist = []
for file in listoflasfiles:
    if file.replace('.las','.tif') not in listofdemfiles:
        outlas =  REMAINING + os.path.basename(file)
        #print(outlas)
        remaininglist.append(outlas)

        shutil.copy(file, outlas)
print(len(remaininglist))

89


In [20]:
demfiles = glob.glob('D:/DitchNet/dem05m/**/*.tif', recursive = True)
print(len(demfiles),' tiffiles were converted')

22046  tiffiles were converted


Some files failed to be converted from las files to dems due to missing lidar points

## High pass median filter

In [ ]:
import sys
import os
sys.path.insert(1, 'Y:/William/DeepLearning/Implementation/code/utils/WBT/')
from whitebox_tools import WhiteboxTools
whitebox_dir = os.path.dirname('Y:/William/DeepLearning/Implementation/code/utils/WBT/')
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(whitebox_dir)

DEMTILES = 'D:/DitchNet/dem05m/'
HPMF = 'D:/DitchNet/highpassmedianfilter/'

for file in os.listdir(DEMTILES):
    if file.endswith('.tif'):
        DemFile = DEMTILES + file
        HighPassMedianFilter = HPMF + file
        args_HPMF = ['--input=' + DemFile, '--output=' + HighPassMedianFilter, '--filterx=11', '--filtery=11', '--sig_digits=2']

        try:
            whitebox.run_tool('HighPassMedianFilter', args_HPMF)
        except:
            print('Unexpected error:', sys.exc_info()[0])
            raise


In [35]:
# list dem files
demfiles = glob.glob('D:/DitchNet/dem05m/**/*.tif', recursive = True)
# list hpmf files
hpmffiles = os.listdir('D:/DitchNet/highpassmedianfilter/')

missingfiles=[]
for file in demfiles:
    head, tail = os.path.split(file)
    if tail not in hpmffiles:
        missingfiles.append(file)
print('The', len((missingfiles)), 'missing files are:', missingfiles)

The 8 missing files are: ['D:/DitchNet/dem05m\\18G002_72925_7250_25.tif', 'D:/DitchNet/dem05m\\18G002_72925_7275_25.tif', 'D:/DitchNet/dem05m\\20B033_66275_3750_25.tif', 'D:/DitchNet/dem05m\\20B033_66275_3775_25.tif', 'D:/DitchNet/dem05m\\20C029_65675_6725_25.tif', 'D:/DitchNet/dem05m\\20D017_67350_4275_25.tif', 'D:/DitchNet/dem05m\\20D026_67925_4900_25.tif', 'D:/DitchNet/dem05m\\20F029_70550_7150_25.tif']


## Attempting to run hpmf on the missing files

In [37]:
# list dem files
demfiles = glob.glob('D:/DitchNet/dem05m/**/*.tif', recursive = True)
# list hpmf files
hpmffiles = os.listdir('D:/DitchNet/highpassmedianfilter/')

missingfiles=[]
for file in demfiles:
    head, tail = os.path.split(file)
    if tail not in hpmffiles:
        missingfiles.append(file)
#print('The', len((missingfiles)), 'missing files are:', missingfiles)


sys.path.insert(1, 'Y:/William/DeepLearning/Implementation/code/utils/WBT/')
from whitebox_tools import WhiteboxTools
whitebox_dir = os.path.dirname('Y:/William/DeepLearning/Implementation/code/utils/WBT/')
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(whitebox_dir)

DEMTILES = 'D:/DitchNet/dem05m/'
#HPMF = 'D:/DitchNet/highpassmedianfilter/'
HPMF = 'D:/DitchNet/test/'
for file in (missingfiles):
    if file.endswith('.tif'):
        #DemFile = DEMTILES + file
        head, tail = os.path.split(file)
        HighPassMedianFilter = HPMF + tail
wbt.high_pass_median_filter(
    i, 
    output, 
    filterx=11, 
    filtery=11, 
    sig_digits=2, 
    callback=default_callback
)


.\whitebox_tools.exe --run="Highpassmedianfilter" --input=D:/DitchNet/dem05m\18G002_72925_7250_25.tif --output=D:/DitchNet/test/18G002_72925_7250_25.tif --filterx=11 --filtery=11 --sig_digits=2 -v

***********************************
* Welcome to HighPassMedianFilter *
***********************************
Reading data...
Binning data: 0%
Binning data: 1%
Binning data: 2%
Binning data: 3%
Binning data: 4%
Binning data: 5%
Binning data: 6%
Binning data: 7%
Binning data: 8%
Binning data: 9%
Binning data: 10%
Binning data: 11%
Binning data: 12%
Binning data: 13%
Binning data: 14%
Binning data: 15%
Binning data: 16%
Binning data: 17%
Binning data: 18%
Binning data: 19%
Binning data: 20%
Binning data: 21%
Binning data: 22%
Binning data: 23%
Binning data: 24%
Binning data: 25%
Binning data: 26%
Binning data: 27%
Binning data: 28%
Binning data: 29%
Binning data: 30%
Binning data: 31%
Binning data: 32%
Binning data: 33%
Binning data: 34%
Binning data: 35%
Binning data: 36%
Binning data: 37%
Binn

These files appear to be too large. Try to convert from las to dem again.First copy the missing laz files into a new directory

In [54]:
import shutil
import glob
import sys
import os

# list dem files
demfiles = glob.glob('D:/DitchNet/dem05m/**/*.tif', recursive = True)
# list hpmf files
#hpmffiles = 'D:/DitchNet/highpassmedianfilter/'
copytodir = 'D:/DitchNet/missinghpmf/'
lazfiles = 'D:/DitchNet/lazfiles/'
missinglazfiles = 'D:/DitchNet/missinglasfiles/'
listoflazfiles = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/**/*.laz', recursive = True)

for file in listoflazfiles:
    head, tail = os.path.split(file)
    if tail.replace('.laz','.tif') in os.listdir(missinglazfiles):
        copylaz = missinglazfiles + tail
        shutil.copy(file, copylaz)
            



Convert laz to las again

In [57]:
from __future__ import print_function
import threading
import os
import sys
import subprocess
import glob


lasdir = 'D:/DitchNet/missinglasfilesagain/'

maxThreads = 60      #threads
activeThreads = 0

class workerThread(threading.Thread):
    def __init__(self, indata):
        threading.Thread.__init__(self)
        self.file = indata

    def run(self):
        global activeThreads
        activeThreads += 1

        #################################
        ##### Start here #####
        #################################
        global lasdir
        lasout = lasdir + os.path.basename(self.file)

        try:
            subprocess.call(['Y:/William/DeepLearning/Implementation/code/utils/laszip', '-i', self.file, '-o', lasout.replace('.laz','.las')])
            print('unpacked ', self.file)
        except:
            print('Unexpected error:', sys.exc_info()[0])
            raise
        #################################
        ##### Stop here #####
        #################################

        activeThreads -= 1

listoflazfiles = glob.glob('D:/DitchNet/missinglasfiles/**/*.laz', recursive = True)
for file in listoflazfiles:
    worker = workerThread(file).start()
    while activeThreads >= maxThreads:
        pass

unpacked  D:/DitchNet/missinglasfiles\18G002_72925_7250_25.laz
unpacked  D:/DitchNet/missinglasfiles\20C029_65675_6725_25.laz
unpacked  D:/DitchNet/missinglasfiles\20D026_67925_4900_25.laz
unpacked  D:/DitchNet/missinglasfiles\18G002_72925_7275_25.laz
unpacked  D:/DitchNet/missinglasfiles\20D017_67350_4275_25.laz
unpacked  D:/DitchNet/missinglasfiles\20F029_70550_7150_25.laz
unpacked  D:/DitchNet/missinglasfiles\20B033_66275_3775_25.laz
unpacked  D:/DitchNet/missinglasfiles\20B033_66275_3750_25.laz


Then run las to dem 

In [58]:
import sys
import os
sys.path.insert(1, 'D:/DitchNet/code/utils/WBT/')
from whitebox_tools import WhiteboxTools
whitebox_dir = os.path.dirname('D:/DitchNet/code/utils/WBT/')
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(whitebox_dir)

whitebox.work_dir = 'D:/DitchNet/missinglasfilesagain/'
whitebox.lidar_tin_gridding(
    i=None,
    output=None,
    parameter='elevation',
    returns='last',
    resolution=0.5,
    exclude_cls='0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18',
    minz=None,
    maxz=None,
    max_triangle_edge_length=None,
    )

.\whitebox_tools.exe --run="LidarTinGridding" --wd="D:/DitchNet/missinglasfilesagain/" --parameter=elevation --returns=last --resolution=0.5 --exclude_cls=0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18 -v

*******************************
* Welcome to LidarTINGridding *
*******************************
Performing interpolation...
Finished 20C029_65675_6725_25 (1 of 8)
Progress: 0%
Finished 20D026_67925_4900_25 (2 of 8)
Progress: 14%
Finished 20D017_67350_4275_25 (3 of 8)
Progress: 28%
Finished 18G002_72925_7250_25 (4 of 8)
Progress: 42%
Finished 20F029_70550_7150_25 (5 of 8)
Progress: 57%
Finished 18G002_72925_7275_25 (6 of 8)
Progress: 71%
Finished 20B033_66275_3775_25 (7 of 8)
Progress: 85%
Finished 20B033_66275_3750_25 (8 of 8)
Progress: 100%
Elapsed Time (including I/O): 39.934s


0

Try to run high pass median filter on remaining files

In [60]:
import sys
import os
sys.path.insert(1, 'Y:/William/DeepLearning/Implementation/code/utils/WBT/')
from whitebox_tools import WhiteboxTools
whitebox_dir = os.path.dirname('Y:/William/DeepLearning/Implementation/code/utils/WBT/')
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(whitebox_dir)

DEMTILES = 'D:/DitchNet/missinglasfilesagain/'
HPMF = 'D:/DitchNet/highpassmedianfilter/'

for file in os.listdir(DEMTILES):
    if file.endswith('.tif'):
        DemFile = DEMTILES + file
        HighPassMedianFilter = HPMF + file
        args_HPMF = ['--input=' + DemFile, '--output=' + HighPassMedianFilter, '--filterx=11', '--filtery=11', '--sig_digits=2']

        try:
            whitebox.run_tool('HighPassMedianFilter', args_HPMF)
        except:
            print('Unexpected error:', sys.exc_info()[0])
            raise


.\whitebox_tools.exe --run="Highpassmedianfilter" --input=D:/DitchNet/missinglasfilesagain/18G002_72925_7250_25.tif --output=D:/DitchNet/missinghpmf/18G002_72925_7250_25.tif --filterx=11 --filtery=11 --sig_digits=2 -v

***********************************
* Welcome to HighPassMedianFilter *
***********************************
Reading data...
Binning data: 0%
Binning data: 1%
Binning data: 2%
Binning data: 3%
Binning data: 4%
Binning data: 5%
Binning data: 6%
Binning data: 7%
Binning data: 8%
Binning data: 9%
Binning data: 10%
Binning data: 11%
Binning data: 12%
Binning data: 13%
Binning data: 14%
Binning data: 15%
Binning data: 16%
Binning data: 17%
Binning data: 18%
Binning data: 19%
Binning data: 20%
Binning data: 21%
Binning data: 22%
Binning data: 23%
Binning data: 24%
Binning data: 25%
Binning data: 26%
Binning data: 27%
Binning data: 28%
Binning data: 29%
Binning data: 30%
Binning data: 31%
Binning data: 32%
Binning data: 33%
Binning data: 34%
Binning data: 35%
Binning data: 36%
B

Check again for missing files

In [61]:
# list dem files
demfiles = glob.glob('D:/DitchNet/dem05m/**/*.tif', recursive = True)
# list hpmf files
hpmffiles = os.listdir('D:/DitchNet/highpassmedianfilter/')

missingfiles=[]
for file in demfiles:
    head, tail = os.path.split(file)
    if tail not in hpmffiles:
        missingfiles.append(file)
print('The', len((missingfiles)), 'missing files are:', missingfiles)

The 0 missing files are: []


all dem files were converted to hpmf files. They are moved to the NAS and images smaller than x pixels where moved

In [62]:
import os
import shutil
from osgeo import gdal

hpmffiles = 'Y:/William/DeepLearning/Implementation/highpassmedianfilter/HPMF2018_2020_2021/'
movedfiles = 'Y:/William/DeepLearning/Implementation/highpassmedianfilter/HPMFtosmall2018_2020_2021/'

for tile in os.listdir(hpmffiles):
    if tile.endswith('.tif'):
        infile = hpmffiles + tile
        outfile = movedfiles + tile
        rds = gdal.Open(infile)
        img_width,img_height=rds.RasterXSize,rds.RasterYSize
        if img_width < 1024 or img_height < 1024:
            shutil.move(infile,outfile)
            print('moved file', infile)


22046  tiffiles were converted to high pass median filter


Remove files that are too small

In [1]:
import os
import shutil
from osgeo import gdal

hpmffiles = 'Y:/William/DeepLearning/Implementation/highpassmedianfilter/HPMF2018_2020_2021/'

for tile in os.listdir(hpmffiles):
    if tile.endswith('.tif'):
        infile = hpmffiles + tile
        rds = gdal.Open(infile)
        img_width,img_height=rds.RasterXSize,rds.RasterYSize
        if img_width < 1024 or img_height < 1024:
            os.remove(infile)
            print('removed', infile)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'Y:/William/DeepLearning/Implementation/highpassmedianfilter/HPMF2018_2020_2021/18A003_61475_3675_25.tif'

## Remove boarder files
The boarder lidar files do not need to be processed and will be removed from the dataset

In [9]:
import glob
listoflazfiles = glob.glob('D:/DitchNet/lazfiles/**/*.laz', recursive = True)
print(len(listoflazfiles),' lazfiles were located')
import os
import shutil
listoffoldernames = []
listofborderfiles = []
listoffilesnames = []
for file in listoflazfiles:
    name = os.path.basename(os.path.dirname(file))
    if 'border' in name:
        listoffoldernames.append(name)
        listofborderfiles.append(file)
        head,tail = os.path.split(file)
        listoffilesnames.append(tail)
print(len(listofborderfiles), 'files are border files')

#import os
#path=os.path(dirname('D:/DitchNet/lazfiles'))

22109  lazfiles were located
5498 files are border files


5498 files are border files


write script to remove border files from hpmf folder

In [26]:
import os
import shutil
listoffoldernames = []
listofborderfiles = []
listoffilesnames = []
for file in listoflazfiles:
    name = os.path.basename(os.path.dirname(file))
    if 'border' in name:
        listoffoldernames.append(name)
        listofborderfiles.append(file)
        head,tail = os.path.split(file)
        listoffilesnames.append(tail)
print(listoffoldernames[1])
print(listofborderfiles[1])
print(listoffilesnames[1])
print(len(listofborderfiles), 'files are border files')
hpmf = 'Y:/William/DeepLearning/Implementation/highpassmedianfilter/HPMF2018_2020_2021/'
hpmfout = 'Y:/William/DeepLearning/Implementation/highpassmedianfilter/HPMFWithoutBoarderFiles/'
for file in os.listdir(hpmf):
    if file.replace('.tif','.laz') not in listoffilesnames:
        hpmffiles = hpmf + file
        hpmfcopy = hpmfout + file
        #os.remove(hpmffiles)
        shutil.copy(hpmffiles, hpmfcopy)

border
D:/DitchNet/lazfiles\2018\18A003\border\18A003_61475_3700_25.laz
18A003_61475_3700_25.laz
5498 files are border files


The copied files were moved to Gullbyn for inference

## Inference results



In [36]:
classified2018 = os.listdir('Y:/William/DeepLearning/Implementation/inference/classified')
print(len(classified2018))
classified2018.insert(0,'classified')
print(classified2018[1])
#72975_6275_25

12942
19A001_61300_3850_25.tif


Copy none-border tiles

In [ ]:
import os
import shutil
from osgeo import gdal

probability = 'G:/DitchNet/Combined/Probability/'
copyprobability = 'G:/DitchNet/inference/probability/'
for tile in os.listdir(probability):
    if tile.endswith('.tif'):
        infile = probability + tile
        outfile = copyprobability + tile
        rds = gdal.Open(infile)
        img_width,img_height=rds.RasterXSize,rds.RasterYSize
        if img_width == 5000 and img_height == 5000:
            shutil.copy(infile, outfile)
           


classified = 'G:/DitchNet/Combined/Classified/'
copyclassified = 'G:/DitchNet/inference/classified/'
for tile in os.listdir(classified):
    if tile.endswith('.tif'):
        infile = classified + tile
        outfile = copyclassified + tile
        rds = gdal.Open(infile)
        img_width,img_height=rds.RasterXSize,rds.RasterYSize
        if img_width == 5000 and img_height == 5000:
            shutil.copy(infile, outfile)

In [37]:
with open('Y:/William/DeepLearning/Implementation/inference/2018.txt', 'w') as f:
    for item in classified2018:
        f.write("%s\n" % item[7:].replace('.tif',''))

## Connect laserdataskog tiles with old lidarsquares by names
New file is named: 21C036_66375_6150_25.tif
Corresponding old square is named: 663_61_7550
Discard the first and last 7 characters and keep 663 and swap position of 61 and 75. 
Make text file with list of old squares mapped by new lidar.

In [54]:
listoflazfiles = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/**/*.laz', recursive = True)
modifiednames = []

for file in listoflazfiles:
    head,tail = os.path.split(file)
    tail = tail[7:-7] # Discard the first and last 7 characters
    name = tail[:3] + '_' + tail[6:8] + '_' + tail[3:5] + tail[8:] # swap position of index 3:5 and 6:8 and add '_' 
    modifiednames.append(name)
header = "nameslaz"
with open('Y:/William/DeepLearning/Implementation/inference/oldtilenamesofnewlidarscannedfiles.txt', 'w') as f:
    f.write(header + "\n")
    f.close
with open('Y:/William/DeepLearning/Implementation/inference/oldtilenamesofnewlidarscannedfiles.txt', 'a') as f:
    for item in modifiednames:
        f.write("%s\n" % item)

Make list of tiles with classified ditches

In [50]:
listofditchfiles = glob.glob('Y:/William/DeepLearning/Implementation/inference/classified/**/*.tif', recursive = True)
modifiednames = []

for file in listofditchfiles:
    head,tail = os.path.split(file)
    tail = tail[7:-7] # Discard the first and last 7 characters
    name = tail[:3] + '_' + tail[6:8] + '_' + tail[3:5] + tail[8:] # swap position of index 3:5 and 6:8 and add '_' 
    modifiednames.append(name)
header = "ditchtiles"
with open('Y:/William/DeepLearning/Implementation/inference/oldtilenamesofclassifiedditchtiles.txt', 'w') as f:
    f.write(header + "\n")
    f.close
with open('Y:/William/DeepLearning/Implementation/inference/oldtilenamesofclassifiedditchtiles.txt', 'a') as f:
    for item in modifiednames:
        f.write("%s\n" % item)

# Solve missing LAS tiles
10170 tiles were mapped correctly

In [6]:
import glob
listoflasfiles = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/lasfiles/**/*.las', recursive = True)
print(len(listoflasfiles),' las files are located in Y:/William/DeepLearning/Implementation/LiDAR/lasfiles')

35843  las files are located in Y:/William/DeepLearning/Implementation/LiDAR/lasfiles


Some of the las files are border files. I need to identify the tiles which are not borde tiles and not classified ditches. 
Make two lists of file names and compare them. 

In [ ]:
#List of ditch tiles
import glob
listofditchfiles = glob.glob('G:/DitchNet/2021_inference/classified/**/*.tif', recursive = True)
ditchnames = []

for file in listofditchfiles:
    head,tail = os.path.split(file)
    ditchnames.append(tail.replace('.tif',''))
    
with open('Y:/William/DeepLearning/Implementation/investigatemissingtiles/ditchnames.txt', 'w') as f:
    for item in ditchnames:
        f.write("%s\n" % item)

Read text file into python list by row

In [6]:
#open file
import csv
ditchlistfile = []
with open('Y:/William/DeepLearning/Implementation/investigatemissingtiles/realditchnames.txt','r') as file:
    for line in file:

        line = line.strip()
        ditchlistfile.append(line)

print(ditchlistfile[0:10])

['18A003_61500_3675_25', '18A003_61500_3700_25', '18A003_61500_3725_25', '18A003_61500_3750_25', '18A003_61500_3775_25', '18A003_61500_3800_25', '18A003_61500_3825_25', '18A003_61500_3850_25', '18A003_61500_3875_25', '18A003_61500_3900_25']


List las files. Some las files are border files and need to be excluded. I need a list of las files that are not border files.

In [5]:
import os
import glob
listoflazfiles = glob.glob('Y:/William/DeepLearning/Implementation/LiDAR/Laserdata_Skog/**/*.laz', recursive = True)
print(len(listoflazfiles),' lazfiles were located')

listofnonfoldernames = [] # border files are located in folders named border
listofnonborderfiles = []
listofnonborderfilesnames = []
for file in listoflazfiles:
    name = os.path.basename(os.path.dirname(file))
    if 'border' not in name:
        listofnonfoldernames.append(name)
        listofnonborderfiles.append(file)
        head,tail = os.path.split(file)
        listofnonborderfilesnames.append(tail.replace('.laz',''))
print(len(listofnonborderfiles), 'files are not border files')
#example
print(listofnonborderfilesnames[0])


35843  lazfiles were located
26939 files are not border files
18A003_61500_3675_25


# find and copy missing lasfiles

Make a list of las filenames that are not border files

In [36]:
import glob
import os

listoflasfilenames = []
for file in listofnonborderfilesnames:
    head,tail=os.path.split(file)
    listoflasfilenames.append(tail.replace('.las',''))
print(len(listoflasfilenames))
print(len(ditchlistfile))


26939
26725


Compare list of file names with list of mapped ditches

In [37]:
listofmissingfiles =[]
for file in listoflasfilenames:
    if file not in ditchlistfile:
        listofmissingfiles.append(file)
print(len(listofmissingfiles))


335


Copy the missing las files to a new dir. 

In [38]:
import shutil
for file in listofmissingfiles:
    inlasfile = 'Y:/William/DeepLearning/Implementation/LiDAR/lasfiles/' + file + '.las'
    outlasfile = 'Y:/William/DeepLearning/Implementation/investigatemissingtiles/missinglasfiles/'
    shutil.copy(inlasfile,outlasfile)
    
    

Convert las files to dem files

In [39]:
import sys
import os
sys.path.insert(1, 'D:/DitchNet/code/utils/WBT/')
from whitebox_tools import WhiteboxTools
whitebox_dir = os.path.dirname('D:/DitchNet/code/utils/WBT/')
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(whitebox_dir)

whitebox.work_dir = 'Y:/William/DeepLearning/Implementation/investigatemissingtiles/missinglasfiles/'
whitebox.lidar_tin_gridding(
    i=None,
    output=None,
    parameter='elevation',
    returns='last',
    resolution=0.5,
    exclude_cls='0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18',
    minz=None,
    maxz=None,
    max_triangle_edge_length=None,
    )

.\whitebox_tools.exe --run="LidarTinGridding" --wd="Y:/William/DeepLearning/Implementation/investigatemissingtiles/missinglasfiles/" --parameter=elevation --returns=last --resolution=0.5 --exclude_cls=0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18 -v

*******************************
* Welcome to LidarTINGridding *
*******************************
Performing interpolation...
Finished 19B024_65300_3650_25 (1 of 335)
Progress: 0%
Finished 18H004_72925_8425_25 (2 of 335)
Finished 19B027_65500_3975_25 (3 of 335)
Finished 19B027_65475_3975_25 (4 of 335)
Finished 19B027_65525_3975_25 (5 of 335)
Progress: 1%
Finished 19B027_65325_4000_25 (6 of 335)
Finished 19B027_65375_3975_25 (7 of 335)
Finished 18H001_72475_8250_25 (8 of 335)
Progress: 2%
Finished 18H001_72450_8250_25 (9 of 335)
Finished 18H001_72475_8200_25 (10 of 335)
Finished 19D021_67250_5450_25 (11 of 335)
Finished 19B027_65675_3975_25 (12 of 335)
Progress: 3%
Finished 19B030_65875_4100_25 (13 of 335)
Finished 19D021_67075_5325_25 (14 of 

0

Convert DEMs to high pass median filter

In [40]:
sys.path.insert(1, 'Y:/William/DeepLearning/Implementation/code/utils/WBT/')
from whitebox_tools import WhiteboxTools
whitebox_dir = os.path.dirname('Y:/William/DeepLearning/Implementation/code/utils/WBT/')
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(whitebox_dir)


DEMTILES = 'Y:/William/DeepLearning/Implementation/investigatemissingtiles/missingdemfiles/'
HPMF = 'Y:/William/DeepLearning/Implementation/investigatemissingtiles/missinghpmffiles/'
for file in os.listdir(DEMTILES):
    if file.endswith('.tif'):
        DemFile = DEMTILES + file
        HighPassMedianFilter = HPMF + file
        args_HPMF = ['--input=' + DemFile, '--output=' + HighPassMedianFilter, '--filterx=11', '--filtery=11', '--sig_digits=2']

        try:
            whitebox.run_tool('HighPassMedianFilter', args_HPMF)
        except:
            print('Unexpected error:', sys.exc_info()[0])
            raise

.\whitebox_tools.exe --run="Highpassmedianfilter" --input=Y:/William/DeepLearning/Implementation/investigatemissingtiles/missingdemfiles/18A003_61600_3600_25.tif --output=Y:/William/DeepLearning/Implementation/investigatemissingtiles/missinghpmffiles/18A003_61600_3600_25.tif --filterx=11 --filtery=11 --sig_digits=2 -v

***********************************
* Welcome to HighPassMedianFilter *
***********************************
Reading data...
Binning data: 0%
Binning data: 1%
Binning data: 2%
Binning data: 3%
Binning data: 4%
Binning data: 5%
Binning data: 6%
Binning data: 7%
Binning data: 8%
Binning data: 9%
Binning data: 10%
Binning data: 11%
Binning data: 12%
Binning data: 13%
Binning data: 14%
Binning data: 15%
Binning data: 16%
Binning data: 17%
Binning data: 18%
Binning data: 19%
Binning data: 20%
Binning data: 21%
Binning data: 22%
Binning data: 23%
Binning data: 24%
Binning data: 25%
Binning data: 26%
Binning data: 27%
Binning data: 28%
Binning data: 29%
Binning data: 30%
Binning